## 1. import the neccesary libraries

In [53]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer

from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

## Question 1: Downloading the data

In [32]:
trips_jaunary = pd.read_parquet('yellow_tripdata_2023-01.parquet')
trips_jaunary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

## Question 2: Computing duration

In [33]:
trips_jaunary["duration"] = trips_jaunary["tpep_dropoff_datetime"] - trips_jaunary["tpep_pickup_datetime"]
trips_jaunary["duration"] = trips_jaunary["duration"].dt.total_seconds() / 60
trips_jaunary["duration"].std()

42.59435124195458

## Question 3: Dropping outliers

In [37]:
total_trips = int(trips_jaunary.shape[0])

trips_january_without_outliers = trips_jaunary[(trips_jaunary["duration"] >= 1) & (trips_jaunary["duration"] <= 60)]

print(f"% left atfer dropped the outliers={trips_january_without_outliers.shape[0]/total_trips * 100:.3f}")

% left atfer dropped the outliers=98.122


## Question 4: One-hot enconding

In [43]:
trips_january_without_outliers.loc[:, 'PULocationID'] = trips_january_without_outliers['PULocationID'].astype(str)
trips_january_without_outliers.loc[:, 'DOLocationID'] = trips_january_without_outliers['DOLocationID'].astype(str)

train_dict = trips_january_without_outliers[['PULocationID', 'DOLocationID']].to_dict(orient='records')

In [47]:
dv = DictVectorizer()
x_train = dv.fit_transform(train_dict)

x_train.shape

(3009173, 515)

## Question 5: Training a Model

In [54]:
y_train = trips_january_without_outliers['duration'].values

lr = LinearRegression()
lr.fit(x_train, y_train)

y_pred = lr.predict(x_train)

RMS on train: 58.51120798407011


In [55]:
print("RMS on train:", mean_squared_error(y_train, y_pred, squared=False))

RMS on train: 7.6492619241381785


/Users/viarias/mlops-zoomcamp/venv/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


## Questiion 6: Evaluating the model

In [61]:
february_trips = pd.read_parquet('yellow_tripdata_2023-02.parquet')

february_trips.loc[:, 'PULocationID'] = february_trips['PULocationID'].astype(str)
february_trips.loc[:, 'DOLocationID'] = february_trips['DOLocationID'].astype(str)

february_trips["duration"] = february_trips["tpep_dropoff_datetime"] - february_trips["tpep_pickup_datetime"]
february_trips["duration"] = february_trips["duration"].dt.total_seconds() / 60
february_trips = february_trips[(february_trips["duration"] >= 1) & (february_trips["duration"] <= 60)]


X_val = february_trips[['PULocationID', 'DOLocationID']].to_dict(orient='records')
y_val = february_trips['duration'].values

/var/folders/gb/m5ldtjhj23x501z2nnbkfwd9xfhpdq/T/ipykernel_23302/139110911.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['142' '71' '71' ... '158' '79' '161']' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  february_trips.loc[:, 'PULocationID'] = february_trips['PULocationID'].astype(str)
/var/folders/gb/m5ldtjhj23x501z2nnbkfwd9xfhpdq/T/ipykernel_23302/139110911.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['163' '71' '71' ... '143' '162' '140']' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  february_trips.loc[:, 'DOLocationID'] = february_trips['DOLocationID'].astype(str)


In [62]:
X_val = dv.transform(X_val)

y_pred = lr.predict(X_val)

print("RMS on validation:", mean_squared_error(y_val, y_pred, squared=False))

RMS on validation: 7.811818380595618


/Users/viarias/mlops-zoomcamp/venv/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
